In [2]:
import pandas as pd

In [51]:
archive_location = 'archive'

filenames = {
    "customers" : "olist_customers_dataset.csv", #Information about customers and their geographic data.
    "geolocation" : "olist_geolocation_dataset.csv",
    "order_items" : "olist_order_items_dataset.csv", #Details about each item in an order, including price, freight value, and the connection between orders and products.
    "order_payments" : "olist_order_payments_dataset.csv", #Payment information associated with each order.
    "order_reviews" : "olist_order_reviews_dataset.csv", #Customer reviews and ratings for orders.
    "orders" : "olist_orders_dataset.csv", #Contains order information such as order status, purchase date, and delivery date.
    "products" : "olist_products_dataset.csv", #Contains product-related information.
    "sellers" : "olist_sellers_dataset.csv", #Details about sellers and their geographic information.
    "product_category_name_translation" : "product_category_name_translation.csv",
}
customers_df = pd.read_csv(f'{archive_location}/{filenames["customers"]}')
geolocation_df = pd.read_csv(f'{archive_location}/{filenames["geolocation"]}')
order_items = pd.read_csv(f'{archive_location}/{filenames["order_items"]}')
order_payments_df = pd.read_csv(f'{archive_location}/{filenames["order_payments"]}')
order_reviews = pd.read_csv(f'{archive_location}/{filenames["order_reviews"]}')
orders_df = pd.read_csv(f'{archive_location}/{filenames["orders"]}')
products = pd.read_csv(f'{archive_location}/{filenames["products"]}')
sellers = pd.read_csv(f'{archive_location}/{filenames["sellers"]}')
product_category_name_translation = pd.read_csv(f'{archive_location}/{filenames["product_category_name_translation"]}')

# print(f"Count of rows before dedublication: {str(len(geolocation_df.index))}")
# since the only option to connect geolocation tables with other tables is geolocation_city	geolocation_state	geolocation_zip_code_prefix I am using this as a subset 
geolocation_df.drop_duplicates(subset = ["geolocation_city","geolocation_zip_code_prefix", "geolocation_state"], inplace=True)
geolocation_df.reset_index(inplace=True , drop=True)
# print(f"Count of rows after dedublication: {len(geolocation_df.index)}")

# print(orders_df[orders_df["order_approved_at"].isna()].head())
orders_df["order_delivered_carrier_date"] = orders_df["order_delivered_carrier_date"].fillna(pd.NaT)
orders_df["order_approved_at"] = orders_df["order_approved_at"].fillna(pd.NaT)
orders_df["order_delivered_customer_date"] = orders_df["order_delivered_customer_date"].fillna(pd.NaT)
# print(orders_df[orders_df["order_approved_at"].isna()].head())


# print(products[products["product_category_name"].isna()].head())
products["product_category_name"] = products["product_category_name"].fillna('')
products["product_name_lenght"] = products["product_category_name"].fillna('')
products["product_description_lenght"] = products["product_description_lenght"].fillna(0)
products["product_photos_qty"] = products["product_photos_qty"].fillna(0)
products["product_weight_g"] = products["product_weight_g"].fillna('')
products["product_length_cm"] = products["product_length_cm"].fillna('')
products["product_photos_qty"] = products["product_photos_qty"].fillna('')
products["product_width_cm"] = products["product_width_cm"].fillna('')
# print('\n\n----------------')
# print(products[products["product_category_name"] == ''].head())


# print(order_reviews[order_reviews["review_comment_message"].isna()].head())
order_reviews["review_comment_title"] = order_reviews["review_comment_title"].fillna('')
order_reviews["review_comment_message"] = order_reviews["review_comment_message"].fillna('')
# print(order_reviews[order_reviews["review_comment_message"] == ''].head())


# print("Calculating total_price")
order_items["total_price"] = order_items["freight_value"] +  order_items["price"]
# print(order_items[["freight_value","price","total_price"]].head())


# print("\n\nCalculating delivery_time")
orders_df["delivery_time"] = pd.to_datetime(orders_df["order_delivered_customer_date"]) - pd.to_datetime(orders_df["order_purchase_timestamp"])
# print(orders_df[["order_delivered_customer_date","order_purchase_timestamp","delivery_time"]].head())

# print("\n\nAdding delivery_time to order_items")
order_items = pd.merge(order_items, orders_df[['order_id', 'delivery_time']], on='order_id', how='left')
# print(order_items['delivery_time'].head())

#tbc on negative values
# print("\n\nCalculating profit_margin")
order_items["profit_margin"] = order_items["price"] - order_items["freight_value"]
# print(order_items[["price","freight_value","profit_margin"]].sort_values(by=["profit_margin"],ascending=True).head())

# print("\n\nCalculating profit_margin")
payments_count = order_payments_df.groupby('order_id').size().reset_index(name='payments_count')
orders_df = orders_df.merge(payments_count, on='order_id', how='left')
orders_df['payments_count'] = orders_df['payments_count'].fillna(0)
orders_df['payments_count'] = orders_df['payments_count'].astype(int)

# print(orders_df['payments_count'].head())

orders_with_price_df = pd.merge(orders_df, order_items[['order_id', 'total_price']], on='order_id', how='left')

orders_with_price_df['running_total'] = orders_with_price_df.groupby('customer_id')['total_price'].cumsum()

#to show the running total of product price for each customer
# testorders_with_price_df = orders_with_price_df[orders_with_price_df["customer_id"] == '1617b1357756262bfa56ab541c47bc16']
# print(testorders_with_price_df[["customer_id","total_price","running_total"]].head())


orders_df['delivery_time'] = pd.to_numeric(orders_df['delivery_time'], errors='coerce')

orders_with_productId_df = pd.merge(orders_df, order_items[['order_id', 'product_id']], on='order_id', how='left')
# print(orders_with_productId_df.head())
orders_with_product_category_df = pd.merge(orders_with_productId_df, products[['product_id', 'product_category_name']], on='product_id', how='left')
# print(orders_with_product_category_df.head())

orders_with_product_category_df['rolling_average_delivery_time'] = orders_with_product_category_df.groupby('product_category_name')['delivery_time'].rolling(window=2).mean().reset_index(level=0, drop=True)
orders_with_product_category_df['delivery_time'] = pd.to_timedelta(orders_with_product_category_df['delivery_time'])
orders_with_product_category_df['rolling_average_delivery_time'] = pd.to_timedelta(orders_with_product_category_df['rolling_average_delivery_time'])

orders_df['delivery_time'] = pd.to_timedelta(orders_df['delivery_time'])

#to show the rolling of the avg delivery time
# testorders_with_product_category_df = orders_with_product_category_df[orders_with_product_category_df["product_category_name"] == "utilidades_domesticas"]
# print(testorders_with_product_category_df[['product_category_name',"delivery_time","rolling_average_delivery_time"]].head())

In [13]:
#add customer_id to Order Items table to connect with Customers's table

db_order_items = pd.merge(order_items, orders_df[['order_id', 'customer_id']], on='order_id', how='left')
#rearange columns for better visibility
db_order_items = db_order_items[["order_id",	"order_item_id",	"product_id",	"seller_id",	"customer_id",	"shipping_limit_date",	"price",	"freight_value",	"total_price",	"profit_margin",	"delivery_time"]]

In [40]:
#add  geolocation_lat	geolocation_lng to Customers's table

db_customers_df = pd.merge(
    customers_df,
    geolocation_df,
    left_on=['customer_zip_code_prefix', 'customer_city', 'customer_state'],
    right_on=['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'],
    how='left'  # Use 'left' to keep all records from customers_df
)

db_customers_df = db_customers_df[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng']]

In [41]:
db_products = products

In [44]:
db_sellers =  pd.merge(
    sellers,
    geolocation_df,
    left_on=['seller_zip_code_prefix', 'seller_city', 'seller_state'],
    right_on=['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'],
    how='left'  # Use 'left' to keep all records from customers_df
)

db_sellers = db_sellers[["seller_id", "seller_zip_code_prefix", "seller_city", "seller_state", "geolocation_lat", "geolocation_lng"]]

In [55]:
db_order_purchase_date = orders_df[["order_purchase_timestamp","order_delivered_customer_date","delivery_time"]]
db_order_purchase_date = db_order_purchase_date.drop_duplicates()

In [60]:
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=LH;"
    "Trusted_Connection=yes;"
)
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')


db_order_items.to_sql('db_order_items', engine, if_exists='replace', index=False)
db_customers_df.to_sql('db_customers_df', engine, if_exists='replace', index=False)
db_products.to_sql('db_products', engine, if_exists='replace', index=False)
db_sellers.to_sql('db_sellers', engine, if_exists='replace', index=False)
db_order_purchase_date.to_sql('db_order_purchase_date', engine, if_exists='replace', index=False)

print("DataFrames successfully uploaded to SQL Server!")

C:\Users\kevin\AppData\Local\Temp\ipykernel_18744\2296086294.py:27: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  db_order_items.to_sql('db_order_items', engine, if_exists='replace', index=False)
C:\Users\kevin\AppData\Local\Temp\ipykernel_18744\2296086294.py:31: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  db_order_purchase_date.to_sql('db_order_purchase_date', engine, if_exists='replace', index=False)


DataFrames successfully uploaded to SQL Server!
